In [ ]:
from functions import download_youtube
video_id = 'ek05M8eCk7M'
#video_id = 'I-cxigjLG0I'
youtube_url = f"https://www.youtube.com/watch?v={video_id}"
job_id = 'Hanni'
file_path, file_name =  download_youtube(youtube_url, job_id=job_id)
file_name

In [ ]:
from functions import slice_video
slice_video(file_name, [(0, 240, 'hanni.mp4')], height=540, width=960)

### OpenCV 설치
- `pip uninstall opencv-python`
- `pip install opencv-contrib-python`

### Face Detection & Recognition

In [ ]:
base_image = "../face_recognition/test_hanni2.jpg"
target_video = "hanni.mp4"

import cv2
import face_recognition

# Load the known image and encode it
known_image = face_recognition.load_image_file(base_image)
known_face_encoding = face_recognition.face_encodings(known_image)[0]

# Load the video file
video_capture = cv2.VideoCapture(target_video)

# Initialize variables
trackers = []
face_names = []
face_distances = []
frame_skip = 36
frame_count = 0
specific_person_present = False  # Flag to indicate if Specific Person is present

def expand_bounding_box(left, top, width, height, frame_width, frame_height, expand_ratio=0.25):
    expand_width = int(width * expand_ratio)
    expand_height = int(height * expand_ratio)
    
    expanded_left = int(max(0, left - expand_width))
    expanded_top = int(max(0, top - expand_height))
    expanded_right = int(min(frame_width, left + width + expand_width))
    expanded_bottom = int(min(frame_height, top + height + expand_height))
    
    return expanded_left, expanded_top, expanded_right, expanded_bottom

def draw_face_annotations(frame, name, distance, bbox, frame_width, frame_height):
    left, top, width, height = bbox
    expanded_left, expanded_top, expanded_right, expanded_bottom = expand_bounding_box(
        left, top, width, height, frame_width, frame_height)
    
    # Convert coordinates to integers
    expanded_left = int(expanded_left)
    expanded_top = int(expanded_top)
    expanded_right = int(expanded_right)
    expanded_bottom = int(expanded_bottom)
    
    # Set color based on name
    if name == "Specific Person":
        color = (0, 0, 255)  # Red
    elif name == "Candidate":
        color = (255, 0, 0)  # Blue
    else:
        color = (0, 255, 0)  # Green
    
    # Draw rectangle
    cv2.rectangle(frame, (expanded_left, expanded_top), (expanded_right, expanded_bottom), color, 2)
    
    # Display name and distance
    cv2.putText(frame, name, (expanded_left, expanded_bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    cv2.putText(frame, f"Distance: {distance:.2f}", (expanded_left, expanded_bottom + 45), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

while True:
    # Read a single frame from the video
    ret, frame = video_capture.read()

    if not ret:
        break

    frame_count += 1

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Get frame dimensions
    frame_height, frame_width, _ = frame.shape

    if not specific_person_present:
        # Attempt face detection in every frame until Specific Person is found
        trackers = []
        face_names = []
        face_distances = []

        # Detect face locations and encodings
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        distances = []
        for face_encoding in face_encodings:
            # Calculate face distance
            distance = face_recognition.face_distance([known_face_encoding], face_encoding)[0]
            distances.append(distance)

        # Set tolerance (e.g., 0.46)
        tolerance = 0.46

        # Find the index of the Specific Person
        specific_person_index = None
        if len(distances) > 0:
            min_distance = min(distances)
            if min_distance < tolerance:
                specific_person_index = distances.index(min_distance)

        if specific_person_index is not None:
            # Specific Person detected
            specific_person_present = True

            for idx, (face_location, face_encoding) in enumerate(zip(face_locations, face_encodings)):
                top, right, bottom, left = face_location

                # Calculate face distance
                distance = distances[idx]

                if distance < tolerance:
                    if idx == specific_person_index:
                        name = "Specific Person"
                    else:
                        name = "Candidate"
                else:
                    name = "Unknown"

                # Initialize tracker
                tracker = cv2.legacy.TrackerKCF_create()
                bbox = (left, top, right - left, bottom - top)
                tracker.init(frame, bbox)
                trackers.append(tracker)
                face_names.append(name)
                face_distances.append(distance)

                # Draw face annotations
                draw_face_annotations(frame, name, distance, bbox, frame_width, frame_height)
        else:
            # Specific Person not detected; do not proceed with tracking
            pass
    else:
        # Specific Person is being tracked
        if frame_count % frame_skip == 0:
            # Reset trackers and face information
            trackers = []
            face_names = []
            face_distances = []

            # Detect face locations and encodings
            face_locations = face_recognition.face_locations(rgb_frame)
            face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

            distances = []
            for face_encoding in face_encodings:
                # Calculate face distance
                distance = face_recognition.face_distance([known_face_encoding], face_encoding)[0]
                distances.append(distance)

            # Set tolerance (e.g., 0.46)
            tolerance = 0.46

            # Find the index of the Specific Person
            specific_person_index = None
            if len(distances) > 0:
                min_distance = min(distances)
                if min_distance < tolerance:
                    specific_person_index = distances.index(min_distance)
                else:
                    specific_person_index = None

            if specific_person_index is not None:
                # Specific Person still detected
                for idx, (face_location, face_encoding) in enumerate(zip(face_locations, face_encodings)):
                    top, right, bottom, left = face_location

                    # Calculate face distance
                    distance = distances[idx]

                    if distance < tolerance:
                        if idx == specific_person_index:
                            name = "Specific Person"
                        else:
                            name = "Candidate"
                    else:
                        name = "Unknown"

                    # Initialize tracker
                    tracker = cv2.legacy.TrackerKCF_create()
                    bbox = (left, top, right - left, bottom - top)
                    tracker.init(frame, bbox)
                    trackers.append(tracker)
                    face_names.append(name)
                    face_distances.append(distance)

                    # Draw face annotations
                    draw_face_annotations(frame, name, distance, bbox, frame_width, frame_height)
            else:
                # Specific Person lost
                specific_person_present = False
                trackers = []
                face_names = []
                face_distances = []
        else:
            # Update all trackers
            new_trackers = []
            new_face_names = []
            new_face_distances = []

            specific_person_still_present = False

            for tracker, name, distance in zip(trackers, face_names, face_distances):
                success, bbox = tracker.update(frame)
                if success:
                    new_trackers.append(tracker)
                    new_face_names.append(name)
                    new_face_distances.append(distance)

                    # Convert bbox to integers
                    bbox = tuple(map(int, bbox))

                    # Draw face annotations
                    draw_face_annotations(frame, name, distance, bbox, frame_width, frame_height)

                    if name == "Specific Person":
                        specific_person_still_present = True
                else:
                    if name == "Specific Person":
                        specific_person_still_present = False

            # Update trackers and face info
            trackers = new_trackers
            face_names = new_face_names
            face_distances = new_face_distances

            if not specific_person_still_present:
                # Specific Person lost during tracking
                specific_person_present = False
                trackers = []
                face_names = []
                face_distances = []

    # Display the video
    cv2.imshow('Video', frame)

    # Exit if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup after finishing
video_capture.release()
cv2.destroyAllWindows()


### Stable Diffusion Web UI API 를 사용하여 Face Swapping

In [ ]:
base_image = "../face_recognition/test_hanni2.jpg"
source_image = "../face_recognition/faces/김태희.jpg"
target_video = "hanni.mp4"

import io
import cv2
import face_recognition
import numpy as np
from get_swapped_face import get_swapped_face

# 특정 인물의 이미지를 로드하여 인코딩
known_image = face_recognition.load_image_file(base_image)
known_face_encoding = face_recognition.face_encodings(known_image)[0]

with open(source_image, "rb") as source_file:
    source_bytes = source_file.read()

# 비디오 파일 로드
video_capture = cv2.VideoCapture(target_video)

# 변수 초기화
trackers = []
face_names = []
face_distances = []
frame_skip = 12
frame_count = 0

# swap_face 함수가 이미 존재한다고 가정합니다.
# def swap_face(jpg_binary):
#     # 얼굴 스왑 처리 후 동일한 크기의 jpg_binary 반환
#     return swapped_jpg_binary

while True:
    # 비디오에서 한 프레임씩 읽기
    ret, frame = video_capture.read()

    if not ret:
        break

    frame_count += 1

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 프레임의 높이와 너비를 가져오기
    frame_height, frame_width, _ = frame.shape

    if frame_count % frame_skip == 0:
        # 추적기와 얼굴 정보 초기화
        trackers = []
        face_names = []
        face_distances = []

        # 얼굴 위치와 인코딩 탐지
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        distances = []
        for face_encoding in face_encodings:
            # 얼굴 거리 계산
            distance = face_recognition.face_distance([known_face_encoding], face_encoding)[0]
            distances.append(distance)

        # 임계값 설정 (예: 0.46)
        tolerance = 0.46

        # 거리 기준으로 가장 작은 인덱스 찾기
        specific_person_index = None
        if len(distances) > 0:
            min_distance = min(distances)
            if min_distance < tolerance:
                specific_person_index = distances.index(min_distance)

        for idx, (face_location, face_encoding) in enumerate(zip(face_locations, face_encodings)):
            top, right, bottom, left = face_location

            # 얼굴 거리 계산
            distance = distances[idx]

            if distance < tolerance:
                if idx == specific_person_index:
                    name = "Specific Person"
                else:
                    name = "Candidate"
            else:
                name = "Unknown"

            # 추적기 초기화
            tracker = cv2.legacy.TrackerKCF_create()
            bbox = (left, top, right - left, bottom - top)
            tracker.init(frame, bbox)
            trackers.append(tracker)
            face_names.append(name)
            face_distances.append(distance)
    else:
        # 모든 추적기 업데이트
        new_trackers = []
        new_face_names = []
        new_face_distances = []

        for tracker, name, distance in zip(trackers, face_names, face_distances):
            success, bbox = tracker.update(frame)
            if success:
                new_trackers.append(tracker)
                new_face_names.append(name)
                new_face_distances.append(distance)
                left, top, width, height = [int(v) for v in bbox]
                right = left + width
                bottom = top + height

                # 사각형을 4방으로 1.5배 확장
                expand_width = int(width * 0.25)  # 원래의 0.25배씩 좌우로 확장
                expand_height = int(height * 0.25)  # 원래의 0.25배씩 상하로 확장

                # 새로운 좌표 계산 (프레임 경계를 넘지 않도록 조정)
                expanded_left = max(0, left - expand_width)
                expanded_top = max(0, top - expand_height)
                expanded_right = min(frame_width, right + expand_width)
                expanded_bottom = min(frame_height, bottom + expand_height)

                # 사각형과 텍스트 표시
                if name == "Specific Person":
                    # 특정 인물에 대해 얼굴 스왑 처리

                    # 확장된 사각형 영역 추출
                    face_region = frame[expanded_top:expanded_bottom, expanded_left:expanded_right]

                    # 이미지를 JPG 바이너리로 인코딩
                    success_enc, jpg_buffer = cv2.imencode('.jpg', face_region)
                    if success_enc:
                        jpg_binary = jpg_buffer.tobytes()

                        # swap_face 함수 호출
                        swap_png_binary = get_swapped_face(source_bytes, jpg_binary)

                        # swap_png_binary가 BytesIO 객체인 경우 처리
                        if isinstance(swap_png_binary, io.BytesIO):
                            swap_png_binary = swap_png_binary.getvalue()

                        # swap_png_binary를 이미지로 디코딩
                        swap_image_array = np.frombuffer(swap_png_binary, np.uint8)
                        swap_image = cv2.imdecode(swap_image_array, cv2.IMREAD_COLOR)

                        # 스왑된 이미지의 크기가 영역과 다를 수 있으므로 리사이즈
                        swap_image = cv2.resize(swap_image, (expanded_right - expanded_left, expanded_bottom - expanded_top))

                        # 스왑된 이미지를 프레임에 적용
                        frame[expanded_top:expanded_bottom, expanded_left:expanded_right] = swap_image

                    # 사각형 그리기
                    color = (0, 0, 255)  # 빨간색
                    cv2.rectangle(frame, (expanded_left, expanded_top), (expanded_right, expanded_bottom), color, 2)
                    # 이름 표시
                    cv2.putText(frame, name, (expanded_left, expanded_bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                    # 거리 표시 (이름 아래)
                    cv2.putText(frame, f"Distance: {distance:.2f}", (expanded_left, expanded_bottom + 45), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

                elif name == "Candidate":
                    # 후보자는 파란색 사각형과 이름 표시
                    color = (255, 0, 0)
                    cv2.rectangle(frame, (expanded_left, expanded_top), (expanded_right, expanded_bottom), color, 2)
                    cv2.putText(frame, name, (expanded_left, expanded_bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                    cv2.putText(frame, f"Distance: {distance:.2f}", (expanded_left, expanded_bottom + 45), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                else:
                    # 다른 인물에 대해 초록색 사각형과 이름만 표시
                    color = (0, 255, 0)
                    cv2.rectangle(frame, (expanded_left, expanded_top), (expanded_right, expanded_bottom), color, 2)
                    cv2.putText(frame, name, (expanded_left, expanded_bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                    cv2.putText(frame, f"Distance: {distance:.2f}", (expanded_left, expanded_bottom + 45), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        # 추적기와 얼굴 정보 업데이트
        trackers = new_trackers
        face_names = new_face_names
        face_distances = new_face_distances

    # 비디오 출력
    cv2.imshow('Video', frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 모든 작업 완료 후 클린업
video_capture.release()
cv2.destroyAllWindows()


### insightface library 를 사용하여 Face Swapping

```
    pip install insightface
    pip install onnxruntime #for CPU-only
    pip install onnxruntime-gpu #For GPU
    pip uninstall opencv-python-headless
    pip install opencv-contrib-python-headless # 또는 opencv-contrib-python
```
- Download `inswapper_128.onnx` & Locate it in a specific directory
- buffalo_l download from : 
    https://github.com/deepinsight/insightface/releases

- unzip buffalo_l.zip on `C:\Users\<user>\.insightface\models\buffalo_l`

In [ ]:
import io
import cv2
import face_recognition
import numpy as np
from inswapper import FaceSwapper

base_image = "../face_recognition/test_hanni2.jpg"
source_image = "../face_recognition/faces/yunsy.jpg"
target_video = "hanni.mp4"

# 특정 인물의 이미지를 로드하여 인코딩
known_image = face_recognition.load_image_file(base_image)
known_face_encoding = face_recognition.face_encodings(known_image)[0]

# FaceSwapper 클래스 인스턴스 생성
face_swapper = FaceSwapper(det_size=(160, 160))

# 소스 얼굴 설정 (face_index는 선택 사항)
success = face_swapper.set_source_face(source_image)

if not success:
    print("소스 얼굴 설정에 실패했습니다.")
    exit()

# 비디오 파일 로드
video_capture = cv2.VideoCapture(target_video)

# 변수 초기화
trackers = []
face_names = []
face_distances = []
frame_skip = 24
frame_count = 0

def process_face(frame, name, distance, bbox, frame_width, frame_height):
    left, top, width, height = bbox
    expand_ratio=0.3
    expand_width = int(width * expand_ratio)
    expand_height = int(height * expand_ratio)

    expanded_left = int(max(0, left - expand_width))
    expanded_top = int(max(0, top - expand_height))
    expanded_right = int(min(frame_width, left + width + expand_width))
    expanded_bottom = int(min(frame_height, top + height + expand_height))

    # 얼굴 영역 추출
    face_region = frame[expanded_top:expanded_bottom, expanded_left:expanded_right]

    if name == "Specific Person":
        # 얼굴 스왑 처리
        swap_image = face_swapper.swap_faces_in_image(face_region)
        if swap_image is not None:
            # 스왑된 이미지를 얼굴 영역 크기에 맞게 조정
            swap_image = cv2.resize(swap_image, (expanded_right - expanded_left, expanded_bottom - expanded_top))
            # 스왑된 얼굴을 프레임에 적용
            frame[expanded_top:expanded_bottom, expanded_left:expanded_right] = swap_image
        color = (0, 0, 255)  # 빨간색
    elif name == "Candidate":
        color = (255, 0, 0)  # 파란색
    else:
        color = (0, 255, 0)  # 초록색

    # 사각형 그리기
    cv2.rectangle(frame, (expanded_left, expanded_top), (expanded_right, expanded_bottom), color, 2)
    # 이름과 거리 표시
    cv2.putText(frame, name, (expanded_left, expanded_bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    cv2.putText(frame, f"Distance: {distance:.2f}", (expanded_left, expanded_bottom + 45), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

while True:
    # 비디오에서 한 프레임씩 읽기
    ret, frame = video_capture.read()

    if not ret:
        break

    frame_count += 1

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 프레임의 높이와 너비를 가져오기
    frame_height, frame_width, _ = frame.shape

    if frame_count % frame_skip == 0:
        # 추적기와 얼굴 정보 초기화
        trackers = []
        face_names = []
        face_distances = []

        # 얼굴 위치와 인코딩 탐지
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        distances = []
        for face_encoding in face_encodings:
            # 얼굴 거리 계산
            distance = face_recognition.face_distance([known_face_encoding], face_encoding)[0]
            distances.append(distance)

        # 임계값 설정 (예: 0.46)
        tolerance = 0.46

        # 거리 기준으로 가장 작은 인덱스 찾기
        specific_person_index = None
        if len(distances) > 0:
            min_distance = min(distances)
            if min_distance < tolerance:
                specific_person_index = distances.index(min_distance)

        for idx, (face_location, face_encoding) in enumerate(zip(face_locations, face_encodings)):
            top, right, bottom, left = face_location
            width = right - left
            height = bottom - top
            bbox = (left, top, width, height)

            # 얼굴 거리 계산
            distance = distances[idx]

            if distance < tolerance:
                if idx == specific_person_index:
                    name = "Specific Person"
                else:
                    name = "Candidate"
            else:
                name = "Unknown"

            # 추적기 초기화
            tracker = cv2.legacy.TrackerKCF_create()
            tracker.init(frame, bbox)
            trackers.append(tracker)
            face_names.append(name)
            face_distances.append(distance)

            # 얼굴 처리 및 주석 그리기
            process_face(frame, name, distance, bbox, frame_width, frame_height)

    else:
        # 모든 추적기 업데이트
        new_trackers = []
        new_face_names = []
        new_face_distances = []

        for tracker, name, distance in zip(trackers, face_names, face_distances):
            success, bbox = tracker.update(frame)
            if success:
                bbox = tuple(map(int, bbox))
                new_trackers.append(tracker)
                new_face_names.append(name)
                new_face_distances.append(distance)

                # 얼굴 처리 및 주석 그리기
                process_face(frame, name, distance, bbox, frame_width, frame_height)

        # 추적기와 얼굴 정보 업데이트
        trackers = new_trackers
        face_names = new_face_names
        face_distances = new_face_distances

    # 비디오 출력
    cv2.imshow('Video', frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 모든 작업 완료 후 클린업
video_capture.release()
cv2.destroyAllWindows()


#### Class 화 & Decorator 사용

In [ ]:
import cv2
import face_recognition
from inswapper import FaceSwapper

class VideoFaceSwapper:
    def __init__(self, base_image, target_video):

        # Load and encode the known image
        known_image = face_recognition.load_image_file(base_image)
        self.known_face_encoding = face_recognition.face_encodings(known_image)[0]
        self.target_video = target_video

    def video_swap(self, func):
        def wrapper(*args, **kwargs):
            # Video capture
            video_capture = cv2.VideoCapture(self.target_video)

            # Initialize variables
            trackers = []
            face_names = []
            face_distances = []
            frame_skip = 24
            frame_count = 0

            while True:
                ret, frame = video_capture.read()
                if not ret:
                    break

                frame_count += 1
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                if frame_count % frame_skip == 0:
                    trackers = []
                    face_names = []
                    face_distances = []

                    face_locations = face_recognition.face_locations(rgb_frame)
                    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

                    distances = []
                    for face_encoding in face_encodings:
                        distance = face_recognition.face_distance([self.known_face_encoding], face_encoding)[0]
                        distances.append(distance)

                    tolerance = 0.46
                    specific_person_index = None
                    if len(distances) > 0:
                        min_distance = min(distances)
                        if min_distance < tolerance:
                            specific_person_index = distances.index(min_distance)

                    for idx, (face_location, face_encoding) in enumerate(zip(face_locations, face_encodings)):
                        top, right, bottom, left = face_location
                        width = right - left
                        height = bottom - top
                        bbox = (left, top, width, height)

                        distance = distances[idx]
                        if distance < tolerance:
                            if idx == specific_person_index:
                                name = "Specific Person"
                            else:
                                name = "Candidate"
                        else:
                            name = "Unknown"

                        # Initialize tracker
                        tracker = cv2.legacy.TrackerKCF_create()
                        tracker.init(frame, bbox)
                        trackers.append(tracker)
                        face_names.append(name)
                        face_distances.append(distance)

                        # Process face and annotate
                        self.annotate_frame(bbox, frame, name, distance, func)
                else:
                    new_trackers = []
                    new_face_names = []
                    new_face_distances = []

                    for tracker, name, distance in zip(trackers, face_names, face_distances):
                        success, bbox = tracker.update(frame)
                        if success:
                            bbox = tuple(map(int, bbox))
                            new_trackers.append(tracker)
                            new_face_names.append(name)
                            new_face_distances.append(distance)

                            # Process face and annotate
                            self.annotate_frame(bbox, frame, name, distance, func)
                        else:
                            print("Tracker lost!")

                    trackers = new_trackers
                    face_names = new_face_names
                    face_distances = new_face_distances

                # Display the video
                cv2.imshow('Video', frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            # Cleanup
            video_capture.release()
            cv2.destroyAllWindows()

        return wrapper

    def annotate_frame(self, bbox, frame, name, distance, func):
        left, top, width, height = bbox
        expand_ratio = 0.3
        expand_width = int(width * expand_ratio)
        expand_height = int(height * expand_ratio)

        expanded_left = int(max(0, left - expand_width))
        expanded_top = int(max(0, top - expand_height))

        frame_height, frame_width, _ = frame.shape
        expanded_right = int(min(frame_width, left + width + expand_width))
        expanded_bottom = int(min(frame_height, top + height + expand_height))

        # Extract face region
        face_region = frame[expanded_top:expanded_bottom, expanded_left:expanded_right]

        if name == "Specific Person":
            # Apply face swap
            swap_image = func(face_region)
            # Replace the face region in the frame
            frame[expanded_top:expanded_bottom, expanded_left:expanded_right] = swap_image
            color = (0, 0, 255)  # Red
        elif name == "Candidate":
            color = (255, 0, 0)  # Blue
        else:
            color = (0, 255, 0)  # Green

        # Draw rectangle and annotations
        cv2.rectangle(frame, (expanded_left, expanded_top), (expanded_right, expanded_bottom), color, 2)
        cv2.putText(frame, name, (expanded_left, expanded_bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        cv2.putText(frame, f"Distance: {distance:.2f}", (expanded_left, expanded_bottom + 45), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

# Initialize the face swapper and known face encoding outside the class
base_image = "../face_recognition/test_hanni2.jpg"
target_video = "hanni.mp4"

# Create an instance of VideoFaceSwapper
swapper = VideoFaceSwapper(base_image, target_video)

# Create FaceSwapper instance
face_swapper = FaceSwapper(det_size=(160, 160))

# Set the source face
source_image = "../face_recognition/faces/yunsy.jpg"
success = face_swapper.set_source_face(source_image)
if not success:
    print("Failed to set source face.")
    exit()

@swapper.video_swap
def swap_other_face(face_region):
    swap_image = face_swapper.swap_faces_in_image(face_region)
    return swap_image if swap_image is not None else face_region

# Start the face-swapping process
swap_other_face()


#### insightface 를 사용한 Face Recognition

- Face Detection

In [ ]:
# 얼굴 인식을 위해 InsightFace를 사용하는 샘플 코드

import cv2
from insightface.app import FaceAnalysis

# FaceAnalysis 객체 초기화 (사전 학습된 모델 사용)
app = FaceAnalysis(name='buffalo_l')  # 'buffalo_l'는 사전 학습된 모델 이름입니다.
app.prepare(ctx_id=-1)  # ctx_id=0은 GPU 사용, ctx_id=-1은 CPU 사용
app.det_model.nms_thresh = 0.4  # NMS 임계값 설정

# 이미지 파일 읽기
img = cv2.imread("../face_recognition/faces/bk.goldengirls01.jpg")  # 처리할 이미지 파일의 경로로 변경하세요.
if img is None:
    raise FileNotFoundError(f"이미지를 불러올 수 없습니다. 경로를 확인하세요")

# 얼굴 검출 및 임베딩 추출
faces = app.get(img)

# 검출된 얼굴 처리
for idx, face in enumerate(faces):
    # 얼굴 영역 표시
    bbox = face.bbox.astype(int)
    cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)
    # 얼굴 임베딩 출력
    #print(f"얼굴 {idx+1} 임베딩 벡터:\n{face.embedding}")

# 결과 이미지 표시
cv2.imshow('얼굴 인식 결과', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

- Face Recognition

In [ ]:
import cv2
import numpy as np
from insightface.app import FaceAnalysis
from sklearn.metrics.pairwise import cosine_similarity

# FaceAnalysis 객체 초기화 (사전 학습된 모델 사용)
app = FaceAnalysis(name='buffalo_l')  # 'buffalo_l'는 사전 학습된 모델 이름입니다.
app.prepare(ctx_id=0)  # ctx_id=0은 GPU 사용, ctx_id=-1은 CPU 사용

# 비교할 기준 얼굴 이미지 로드 및 임베딩 추출
ref_img_path = "../face_recognition/faces/hanni01.jpg"  # 기준 얼굴 이미지 경로로 변경하세요.
ref_img = cv2.imread(ref_img_path)
if ref_img is None:
    raise FileNotFoundError(f"기준 이미지를 불러올 수 없습니다. 경로를 확인하세요: {ref_img_path}")

ref_faces = app.get(ref_img)
if len(ref_faces) == 0:
    raise ValueError("기준 이미지에서 얼굴을 검출하지 못했습니다.")

# 기준 얼굴의 임베딩 추출 (첫 번째 얼굴 사용)
ref_embedding = ref_faces[0].embedding

# 비교할 대상 이미지 로드 및 얼굴 임베딩 추출
target_img_path = "../face_recognition/faces/newjeans02_g.jpg"  # 대상 이미지 경로로 변경하세요.
target_img = cv2.imread(target_img_path)
if target_img is None:
    raise FileNotFoundError(f"대상 이미지를 불러올 수 없습니다. 경로를 확인하세요: {target_img_path}")

target_faces = app.get(target_img)

# target_faces를 x축 기준으로 정렬 (좌에서 우로)
target_faces.sort(key=lambda face: face.bbox[0])

# 검출된 얼굴들에 대해 유사도 계산 및 표시
for idx, face in enumerate(target_faces):
    # 대상 얼굴의 임베딩 추출
    target_embedding = face.embedding

    # 코사인 유사도 계산
    similarity = cosine_similarity([ref_embedding], [target_embedding])[0][0]

    # 유사도 출력
    print(f"얼굴 {idx+1} 유사도: {similarity:.4f}")

    # 얼굴 영역 표시
    bbox = face.bbox.astype(int)
    cv2.rectangle(target_img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)
    # 유사도 텍스트 표시
    cv2.putText(target_img, f"{idx} : {similarity:.2f}", (bbox[0], bbox[1]-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

# 결과 이미지 표시
cv2.imshow('Similarity Result', target_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


#### mp4 파일 만들기 옵션 추가

In [1]:
import cv2
import numpy as np
import os
from insightface.app import FaceAnalysis
from sklearn.metrics.pairwise import cosine_similarity
from inswapper import FaceSwapper

class VideoFaceSwapper:
    def __init__(self, base_image, target_video, tolerance=0.35, output_video=None, display_video=True, display_rectangle=True, segments=None):
        # Initialize FaceAnalysis object
        self.app = FaceAnalysis(name='buffalo_l')
        self.app.prepare(ctx_id=0)  # 0 Use GPU (set ctx_id=-1 to use CPU)

        # Load the reference face image and extract embedding
        ref_img = cv2.imread(base_image)
        if ref_img is None:
            raise FileNotFoundError(f"Unable to load the reference image: {base_image}")

        ref_faces = self.app.get(ref_img)
        if len(ref_faces) == 0:
            raise ValueError("No faces detected in the reference image.")

        # Extract embedding of the reference face (use the first face)
        self.known_face_embedding = ref_faces[0].embedding
        self.target_video = target_video
        self.output_video = output_video
        self.display_video = display_video
        self.display_rectangle = display_rectangle
        self.tolerance = tolerance
        self.specific_person_present = False  # Flag to indicate if Specific Person is present

        # Segments to process
        self.segments = segments  # List of tuples (start_time_str, duration_in_seconds)

        # If output_video is None, do not use video saving feature
        if self.output_video is not None:
            # Check and create the directory for output_video
            output_dir = os.path.dirname(self.output_video)
            if output_dir and not os.path.exists(output_dir):
                os.makedirs(output_dir)
            # Check file extension
            _, ext = os.path.splitext(self.output_video)
            ext = ext.lower()
            if ext == '.webm':
                self.fourcc = cv2.VideoWriter_fourcc(*'VP90')  # WebM format codec
            else:
                self.fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Default codec

    def video_swap(self, func):
        def wrapper(*args, **kwargs):
            # Video capture
            video_capture = cv2.VideoCapture(self.target_video)

            # Get video properties
            fps = video_capture.get(cv2.CAP_PROP_FPS)
            width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
            total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))  # Total number of frames

            # Set up video writer if output_video is specified
            if self.output_video:
                video_writer = cv2.VideoWriter(self.output_video, self.fourcc, fps, (width, height))

            # Initialize variables
            trackers = []
            face_names = []
            face_similarities = []
            frame_skip = 24
            frame_count = 0

            # Convert segments to list of (start_frame, end_frame)
            if self.segments is not None:
                segment_frames = []
                for start_time_str, duration in self.segments:
                    start_seconds = self._time_str_to_seconds(start_time_str)
                    end_seconds = start_seconds + duration
                    start_frame = int(start_seconds * fps)
                    end_frame = int(end_seconds * fps)
                    segment_frames.append((start_frame, end_frame))
            else:
                # If no segments are provided, process the entire video
                segment_frames = [(0, total_frames)]

            # Process each segment
            for start_frame, end_frame in segment_frames:
                if start_frame >= total_frames:
                    print(f"Start frame {start_frame} exceeds total frames {total_frames}. Skipping segment.")
                    continue

                # Adjust end_frame if it exceeds total_frames
                if end_frame > total_frames:
                    end_frame = total_frames

                # Set video capture to the start frame
                video_capture.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
                frame_count = start_frame

                while frame_count < end_frame:
                    ret, frame = video_capture.read()
                    if not ret:
                        break

                    frame_count += 1

                    print(f"specific_person_present :{self.specific_person_present}")

                    if not self.specific_person_present:
                        # Attempt to detect Specific Person in every frame
                        trackers = []
                        face_names = []
                        face_similarities = []

                        # Detect faces and extract embeddings
                        faces = self.app.get(frame)
                        
                        if len(faces) == 0:
                            continue  # No faces detected, skip to next frame

                        similarities = []
                        for face in faces:
                            face_embedding = face.embedding

                            # Calculate cosine similarity
                            similarity = cosine_similarity(
                                [self.known_face_embedding], [face_embedding]
                            )[0][0]
                            similarities.append(similarity)

                        # Set similarity threshold
                        tolerance = self.tolerance

                        # Find the index of the Specific Person
                        specific_person_index = None
                        if len(similarities) > 0:
                            max_similarity = max(similarities)
                            if max_similarity > tolerance:
                                specific_person_index = similarities.index(max_similarity)

                        #print(f"face count : {len(faces)} similarities : {similarities} specific_person_index : {specific_person_index}")
                        
                        if specific_person_index is not None:
                            # Specific Person detected
                            self.specific_person_present = True

                            for idx, face in enumerate(faces):
                                bbox = face.bbox.astype(int)
                                similarity = similarities[idx]

                                if similarity > tolerance:
                                    if idx == specific_person_index:
                                        name = "Specific Person"
                                    else:
                                        name = "Candidate"
                                else:
                                    name = "Unknown"

                                # Initialize tracker
                                tracker = cv2.legacy.TrackerKCF_create()
                                x1, y1, x2, y2 = bbox
                                w = x2 - x1
                                h = y2 - y1
                                tracker_bbox = (x1, y1, w, h)
                                tracker.init(frame, tracker_bbox)
                                trackers.append(tracker)
                                face_names.append(name)
                                face_similarities.append(similarity)

                                # Annotate frame and apply face swap if needed
                                self.annotate_frame(tracker_bbox, frame, name, similarity, func)
                        else:
                            # Specific Person not detected, process Unknown faces
                            self.specific_person_present = False  # Ensure the flag is False

                            for idx_face, face in enumerate(faces):
                                bbox = face.bbox.astype(int)
                                similarity = similarities[idx_face]

                                if similarity > tolerance:
                                    name = "Candidate"
                                else:
                                    name = "Unknown"

                                # Since we are not tracking, we do not initialize trackers
                                # Annotate frame without applying face swap
                                x1, y1, x2, y2 = bbox
                                w = x2 - x1
                                h = y2 - y1
                                tracker_bbox = (x1, y1, w, h)
                                # Pass func=None to indicate no face swap should be applied
                                self.annotate_frame(tracker_bbox, frame, name, similarity, func=None)
                    else:
                        # Specific Person is being tracked
                        if frame_count % frame_skip == 0:
                            # Re-detect faces
                            trackers = []
                            face_names = []
                            face_similarities = []

                            faces = self.app.get(frame)
                            if len(faces) == 0:
                                self.specific_person_present = False
                                continue

                            similarities = []
                            for face in faces:
                                face_embedding = face.embedding

                                # Calculate cosine similarity
                                similarity = cosine_similarity(
                                    [self.known_face_embedding], [face_embedding]
                                )[0][0]
                                similarities.append(similarity)

                            # Set similarity threshold
                            tolerance = self.tolerance

                            # Find the index of the Specific Person
                            specific_person_index = None
                            if len(similarities) > 0:
                                max_similarity = max(similarities)
                                if max_similarity > tolerance:
                                    specific_person_index = similarities.index(max_similarity)
                                else:
                                    specific_person_index = None

                            if specific_person_index is not None:
                                # Specific Person still detected
                                for idx, face in enumerate(faces):
                                    bbox = face.bbox.astype(int)
                                    similarity = similarities[idx]

                                    if similarity > tolerance:
                                        if idx == specific_person_index:
                                            name = "Specific Person"
                                        else:
                                            name = "Candidate"
                                    else:
                                        name = "Unknown"

                                    # Initialize tracker
                                    tracker = cv2.legacy.TrackerKCF_create()
                                    x1, y1, x2, y2 = bbox
                                    w = x2 - x1
                                    h = y2 - y1
                                    tracker_bbox = (x1, y1, w, h)
                                    tracker.init(frame, tracker_bbox)
                                    trackers.append(tracker)
                                    face_names.append(name)
                                    face_similarities.append(similarity)

                                    # Annotate frame and apply face swap if needed
                                    self.annotate_frame(tracker_bbox, frame, name, similarity, func)
                            else:
                                # Specific Person lost
                                self.specific_person_present = False
                                trackers = []
                                face_names = []
                                face_similarities = []
                        else:
                            # Update trackers
                            new_trackers = []
                            new_face_names = []
                            new_face_similarities = []
                            specific_person_still_present = False

                            for tracker, name, similarity in zip(trackers, face_names, face_similarities):
                                success, tracker_bbox = tracker.update(frame)
                                if success:
                                    tracker_bbox = tuple(map(int, tracker_bbox))
                                    new_trackers.append(tracker)
                                    new_face_names.append(name)
                                    new_face_similarities.append(similarity)

                                    # Annotate frame and apply face swap if needed
                                    self.annotate_frame(tracker_bbox, frame, name, similarity, func)

                                    if name == "Specific Person":
                                        specific_person_still_present = True
                                else:
                                    if name == "Specific Person":
                                        specific_person_still_present = False

                            # Update trackers and face info
                            trackers = new_trackers
                            face_names = new_face_names
                            face_similarities = new_face_similarities

                            if not specific_person_still_present:
                                # Specific Person lost during tracking
                                self.specific_person_present = False
                                trackers = []
                                face_names = []
                                face_similarities = []

                    # Display current frame number / total frames at the top-left corner
                    cv2.putText(frame, f"Frame: {frame_count}/{total_frames}", (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

                    # Output or display video
                    if self.display_video:
                        cv2.imshow('Video', frame)
                        if cv2.waitKey(1) & 0xFF == ord('q'):
                            break

                    if self.output_video:
                        video_writer.write(frame)

            # Cleanup
            video_capture.release()
            if self.output_video:
                video_writer.release()
            if self.display_video:
                cv2.destroyAllWindows()

        return wrapper

    def annotate_frame(self, bbox, frame, name, similarity, func):
        left, top, width, height = map(int, bbox)
        expand_ratio = 0.3
        expand_width = int(width * expand_ratio)
        expand_height = int(height * expand_ratio)

        expanded_left = int(max(0, left - expand_width))
        expanded_top = int(max(0, top - expand_height))

        frame_height, frame_width, _ = frame.shape
        expanded_right = int(min(frame_width, left + width + expand_width))
        expanded_bottom = int(min(frame_height, top + height + expand_height))

        # Extract face region
        face_region = frame[expanded_top:expanded_bottom, expanded_left:expanded_right]

        if name == "Specific Person" and func is not None:
            # Apply face swap
            swap_image = func(face_region)
            # Replace the face region with the swapped image
            if swap_image is not None:
                swap_image_resized = cv2.resize(swap_image, (expanded_right - expanded_left, expanded_bottom - expanded_top))
                frame[expanded_top:expanded_bottom, expanded_left:expanded_right] = swap_image_resized
            color = (0, 0, 255)  # Red
        elif name == "Candidate":
            color = (255, 0, 0)  # Blue
        else:
            color = (0, 255, 0)  # Green

        if self.display_rectangle:
            # Draw rectangle and annotations
            cv2.rectangle(frame, (expanded_left, expanded_top), (expanded_right, expanded_bottom), color, 2)
            cv2.putText(frame, name, (expanded_left, expanded_bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            cv2.putText(frame, f"Similarity: {similarity:.2f}", (expanded_left, expanded_bottom + 45), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    def _time_str_to_seconds(self, time_str):
        # Convert "mm:ss" format to total seconds
        minutes, seconds = map(int, time_str.split(':'))
        total_seconds = minutes * 60 + seconds
        return total_seconds

# Set the base image and target video
base_image = "../face_recognition/test_hanni2.jpg"
target_video = "hanni.mp4"

# Define the segments to process (list of tuples with start time and duration in seconds)
segments = [("00:00", 10), ("01:00", 15)]  # Process from 0:00 for 10 seconds, and from 1:00 for 5 seconds

# Create an instance of VideoFaceSwapper with the segments
swapper = VideoFaceSwapper(base_image, target_video, output_video="output.webm", display_video=True, display_rectangle=True, segments=segments)
# If output_video ends with '.webm', it will be saved in WebM format.

# Create an instance of FaceSwapper
face_swapper = FaceSwapper(det_size=(320, 320))

# Set the source face
source_image = "../face_recognition/faces/kimhs.jpg"
success = face_swapper.set_source_face(source_image)
if not success:
    print("Failed to set source face.")
    exit()

@swapper.video_swap
def swap_other_face(face_region):
    swap_image = face_swapper.swap_faces_in_image(face_region)
    return swap_image if swap_image is not None else face_region

# Start the face swap process
swap_other_face()


INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.15 (you have 1.4.11). Upgrade using: pip install --upgrade albumentations
c:\Python310\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:65: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\tanmi/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\tanmi/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\tanmi/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\tanmi/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\tanmi/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.

c:\Python310\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\tanmi/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\tanmi/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\tanmi/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\tanmi/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\tanmi/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.

c:\Python310\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


...

specific_person_present :True
specific_person_present :True
specific_person_present :True
specific_person_present :True
specific_person_present :True
specific_person_present :True
specific_person_present :True
specific_person_present :True
specific_person_present :True
specific_person_present :True
specific_person_present :True
specific_person_present :True
specific_person_present :True
specific_person_present :True
specific_person_present :True
specific_person_present :True
specific_person_present :True
specific_person_present :True
specific_person_present :True
specific_person_present :True
specific_person_present :True
specific_person_present :True
specific_person_present :True
specific_person_present :True
specific_person_present :False


c:\Python310\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


face count : 3 similarities : [0.1889267, 0.36581963, 0.20342335] specific_person_index : 1


c:\Python310\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


specific_person_present :True
